## Setup Requirements

In [4]:
%%capture

import json
import virustotal3.core as vt3core
from dotenv import load_dotenv
import os
import csv
import time
import datetime

load_dotenv()

VT_API = os.getenv('VT_API')
vtcore_files = vt3core.Files(VT_API)
vtcore_domains = vt3core.Domains(VT_API)

collected_data_dir = './collected_data'
input_file = './indicators.csv'
timeline_csv = './timeline.csv'
dates = []

## Get Data from VirusTotal

In [6]:
# Read indicator records from input CSV file
csv_file = open(input_file, 'r')
csv_file_records = csv.DictReader(csv_file)

for record in csv_file_records:
    record_type = record['Type']
    record_value = record['Value']
    
    record_out_path = '{0}/{1}.json'.format(collected_data_dir, record_value)
    
    print("Getting data from VirusTotal for " + record_value)

    # Query VT and save indicator info if data does not exist
    if(os.path.exists(record_out_path) == False):
        
        if(record_type == 'File'):
            record_info = vtcore_files.info_file(record_value)
        elif(record_type == 'Host'):
            record_info = vtcore_domains.info_domain(record_value)
        
        with open(record_out_path, 'w+') as out_data:
            out_data.write(json.dumps(record_info, indent=2))

    # Parse indicator data file for dates
    with open(record_out_path) as record_file:
        record_info_all = json.loads(record_file.read())
        record_info_data = record_info_all['data']['attributes']

        # Add dates for indicator to timeline file
        with open(timeline_csv, 'a+') as timeline_file:
            timeline_writer = csv.writer(timeline_file)

            creation_date = datetime.datetime.fromtimestamp(record_info_data['creation_date'])
            modification_date = datetime.datetime.fromtimestamp(record_info_data['last_modification_date'])


            timeline_writer.writerow([record_value, 'Creation Date', creation_date])
            timeline_writer.writerow([record_value, 'Last Modification Date', modification_date])

            if(record_type == 'File'):
                submission_date = datetime.datetime.fromtimestamp(record_info_data['first_submission_date'])
                timeline_writer.writerow([record_value, 'First Submission Date', submission_date])

    # time.sleep(60)
                    
csv_file.close()

Getting data from VirusTotal for ce21319bd21f76ab0f188a514e8ab1fe6f960c257475e45a23d11125d78df428
Getting data from VirusTotal for avsvmcloud.com


## Print Saved Dates

In [9]:
import pandas as pd

df = pd.read_csv(timeline_csv, sep=",", names=["IOC", "Date Type", "Date"])

df.sort_values(by=['Date'])

,IOC,Date Type,Date
3,avsvmcloud.com,Creation Date,2018-07-25 04:38:29
0,ce21319bd21f76ab0f188a514e8ab1fe6f960c257475e4...,Creation Date,2019-12-23 05:10:01
2,ce21319bd21f76ab0f188a514e8ab1fe6f960c257475e4...,First Submission Date,2021-05-06 06:14:47
1,ce21319bd21f76ab0f188a514e8ab1fe6f960c257475e4...,Last Modification Date,2021-10-03 16:25:05
4,avsvmcloud.com,Last Modification Date,2021-11-07 06:36:55
